In [2]:
!pip install scikit-learn
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 50.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 2.5 MB/s eta 0:00:00eta 0:00:01


In [1]:
import os
import rasterio
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, log_loss

In [4]:
train_files = ['tile00.csv', 'tile01.csv', 'tile02.csv' ,'tile03.csv']
train = pd.concat([pd.read_csv(f) for f in train_files], ignore_index=True)
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)



/tmp/ipykernel_3431/3763426186.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

In [5]:
# Train the Random Forest classifier
clf = RandomForestClassifier(n_estimators=30, random_state=42, verbose=2)
clf.fit(X_train, y_train)


building tree 1 of 30
building tree 2 of 30
building tree 3 of 30
building tree 4 of 30
building tree 5 of 30
building tree 6 of 30
building tree 7 of 30
building tree 8 of 30
building tree 9 of 30
building tree 10 of 30
building tree 11 of 30
building tree 12 of 30
building tree 13 of 30
building tree 14 of 30
building tree 15 of 30
building tree 16 of 30
building tree 17 of 30
building tree 18 of 30
building tree 19 of 30
building tree 20 of 30
building tree 21 of 30
building tree 22 of 30
building tree 23 of 30
building tree 24 of 30
building tree 25 of 30
building tree 26 of 30
building tree 27 of 30
building tree 28 of 30
building tree 29 of 30
building tree 30 of 30


RandomForestClassifier(n_estimators=30, random_state=42, verbose=2)

In [6]:
predictions = clf.predict(X_test)


In [7]:

print(classification_report(y_test, predictions))

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00   3138409
         1.0       0.86      0.20      0.32       487
         2.0       0.90      0.22      0.35       279
         3.0       1.00      1.00      1.00         1
         4.0       0.76      0.20      0.32       185
         5.0       0.75      0.14      0.23        65
         6.0       0.88      0.15      0.26        46
         7.0       0.00      0.00      0.00        10

    accuracy                           1.00   3139482
   macro avg       0.77      0.36      0.43   3139482
weighted avg       1.00      1.00      1.00   3139482



/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
print(np.unique(predictions), np.unique(y_test), np.unique(y_train))

[0. 1. 2. 3. 4. 5. 6.] [0. 1. 2. 3. 4. 5. 6. 7.] [0. 1. 2. 3. 4. 5. 6. 7.]


## Hyperparamter Tuning with GridSearch

In [6]:
from sklearn.model_selection import RandomizedSearchCV
rs_space={'max_depth':[None, 3],
              'n_estimators':[10,20],
              'max_features':[1, 5, 10],
              'criterion':['gini','entropy'],
              'min_samples_leaf':[1, 2, 3, 4],
              'min_samples_split':[2, 5]
          }

rf = RandomForestClassifier()

rf_random = RandomizedSearchCV(rf, rs_space, n_iter=50, n_jobs=-1, cv=3, verbose=1)


In [7]:

model_random = rf_random.fit(X_train, y_train)

# random random search results
print('Best random search hyperparameters are: '+str(model_random.best_params_))
print('Best random search score is: '+str(model_random.best_score_))

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best random search hyperparameters are: {'n_estimators': 10, 'min_samples_split': 2, 'min_samples_leaf': 3, 'max_features': 10, 'max_depth': None, 'criterion': 'entropy'}
Best random search score is: 0.9996869819862434


In [8]:
predictions = rf_random.predict(X_test)
print(classification_report(y_test, predictions))

/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00   3138409
         1.0       0.87      0.16      0.26       487
         2.0       0.94      0.21      0.35       279
         3.0       1.00      1.00      1.00         1
         4.0       0.80      0.15      0.25       185
         5.0       0.75      0.18      0.30        65
         6.0       0.90      0.20      0.32        46
         7.0       0.00      0.00      0.00        10

    accuracy                           1.00   3139482
   macro avg       0.78      0.36      0.44   3139482
weighted avg       1.00      1.00      1.00   3139482



/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [11]:
# predictions = clf.predict(X_test)
# print(classification_report(y_test, predictions))

## 10 samples

In [2]:
train = pd.read_csv('all_tiles_sample1.csv')
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

rf = RandomForestClassifier(n_estimators=10, min_samples_split= 2, min_samples_leaf = 3, max_features=10, max_depth=None, criterion='entropy').fit(X_train, y_train)


y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)
print(log_loss(y_test, y_prob))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))


/tmp/ipykernel_24629/1150241416.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


0.5433369050217051
              precision    recall  f1-score   support

           0       0.90      0.99      0.95     59992
           1       0.92      0.61      0.73      7528
           2       0.96      0.45      0.62      2831
           3       0.97      0.42      0.59       480
           4       0.97      0.48      0.64      1931
           5       0.95      0.45      0.61       604
           6       0.99      0.44      0.61       664
           7       0.94      0.35      0.51       308

    accuracy                           0.91     74338
   macro avg       0.95      0.52      0.66     74338
weighted avg       0.91      0.91      0.89     74338

0.9056202749603164


In [3]:
train = pd.read_csv('all_tiles_sample2.csv')
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

rf = RandomForestClassifier(n_estimators=10, min_samples_split= 2, min_samples_leaf = 3, max_features=10, max_depth=None, criterion='entropy').fit(X_train, y_train)

y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)
print(log_loss(y_test, y_prob))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

/tmp/ipykernel_24629/3813195334.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


0.5179229962405595
              precision    recall  f1-score   support

           0       0.90      0.99      0.95     60004
           1       0.93      0.62      0.74      7531
           2       0.97      0.45      0.62      2829
           3       0.96      0.49      0.65       479
           4       0.98      0.50      0.66      1919
           5       0.97      0.44      0.60       603
           6       0.97      0.45      0.62       662
           7       0.92      0.35      0.51       311

    accuracy                           0.91     74338
   macro avg       0.95      0.54      0.67     74338
weighted avg       0.91      0.91      0.90     74338

0.9078129624149156


In [4]:
train = pd.read_csv('all_tiles_sample3.csv')
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

rf = RandomForestClassifier(n_estimators=10, min_samples_split= 2, min_samples_leaf = 3, max_features=10, max_depth=None, criterion='entropy').fit(X_train, y_train)


y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)
print(log_loss(y_test, y_prob))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

/tmp/ipykernel_24629/2028640509.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


0.5482642053574184
              precision    recall  f1-score   support

           0       0.90      0.99      0.94     59999
           1       0.92      0.59      0.72      7533
           2       0.96      0.42      0.58      2828
           3       0.94      0.43      0.59       479
           4       0.97      0.46      0.62      1925
           5       0.96      0.45      0.61       604
           6       0.98      0.42      0.59       660
           7       0.94      0.38      0.55       310

    accuracy                           0.90     74338
   macro avg       0.95      0.52      0.65     74338
weighted avg       0.91      0.90      0.89     74338

0.9022976136027334


In [5]:
train = pd.read_csv('all_tiles_sample4.csv')
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

rf = RandomForestClassifier(n_estimators=10, min_samples_split= 2, min_samples_leaf = 3, max_features=10, max_depth=None, criterion='entropy').fit(X_train, y_train)


y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)
print(log_loss(y_test, y_prob))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

/tmp/ipykernel_24629/1188003684.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


0.5299013202551536
              precision    recall  f1-score   support

           0       0.90      0.99      0.95     60000
           1       0.92      0.60      0.73      7532
           2       0.96      0.43      0.59      2830
           3       0.95      0.43      0.59       482
           4       0.97      0.49      0.65      1921
           5       0.97      0.43      0.60       605
           6       0.99      0.43      0.60       659
           7       0.92      0.36      0.51       309

    accuracy                           0.90     74338
   macro avg       0.95      0.52      0.65     74338
weighted avg       0.91      0.90      0.89     74338

0.9044903010573327


In [6]:
train = pd.read_csv('all_tiles_sample5.csv')
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

rf = RandomForestClassifier(n_estimators=10, min_samples_split= 2, min_samples_leaf = 3, max_features=10, max_depth=None, criterion='entropy').fit(X_train, y_train)


y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)
print(log_loss(y_test, y_prob))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

/tmp/ipykernel_24629/2103913668.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


0.5281327126727648
              precision    recall  f1-score   support

           0       0.90      1.00      0.94     60002
           1       0.93      0.59      0.72      7519
           2       0.96      0.43      0.59      2833
           3       0.95      0.43      0.60       480
           4       0.96      0.46      0.62      1926
           5       0.98      0.42      0.59       606
           6       0.98      0.43      0.60       663
           7       0.94      0.33      0.49       309

    accuracy                           0.90     74338
   macro avg       0.95      0.51      0.64     74338
weighted avg       0.91      0.90      0.89     74338

0.9027818881325836


In [7]:
train = pd.read_csv('all_tiles_sample6.csv')
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

rf = RandomForestClassifier(n_estimators=10, min_samples_split= 2, min_samples_leaf = 3, max_features=10, max_depth=None, criterion='entropy').fit(X_train, y_train)

y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)
print(log_loss(y_test, y_prob))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

/tmp/ipykernel_24629/2069512487.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


0.5569120553745269
              precision    recall  f1-score   support

           0       0.90      1.00      0.94     60002
           1       0.93      0.58      0.71      7526
           2       0.96      0.42      0.58      2831
           3       0.95      0.45      0.61       477
           4       0.96      0.43      0.60      1926
           5       0.97      0.39      0.56       605
           6       0.99      0.43      0.59       661
           7       0.93      0.34      0.49       310

    accuracy                           0.90     74338
   macro avg       0.95      0.50      0.64     74338
weighted avg       0.90      0.90      0.89     74338

0.8998089806021147


In [8]:
train = pd.read_csv('all_tiles_sample7.csv')
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

rf = RandomForestClassifier(n_estimators=10, min_samples_split= 2, min_samples_leaf = 3, max_features=10, max_depth=None, criterion='entropy').fit(X_train, y_train)


y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)
print(log_loss(y_test, y_prob))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

/tmp/ipykernel_24629/609367186.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


0.568858219272155
              precision    recall  f1-score   support

           0       0.90      0.99      0.94     60005
           1       0.92      0.59      0.72      7529
           2       0.96      0.41      0.57      2832
           3       0.95      0.42      0.58       479
           4       0.96      0.46      0.63      1919
           5       0.97      0.41      0.58       606
           6       0.98      0.42      0.59       659
           7       0.94      0.35      0.51       309

    accuracy                           0.90     74338
   macro avg       0.95      0.51      0.64     74338
weighted avg       0.91      0.90      0.89     74338

0.9012618041916651


In [9]:
train = pd.read_csv('all_tiles_sample8.csv')
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

rf = RandomForestClassifier(n_estimators=10, min_samples_split= 2, min_samples_leaf = 3, max_features=10, max_depth=None, criterion='entropy').fit(X_train, y_train)

y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)
print(log_loss(y_test, y_prob))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

/tmp/ipykernel_24629/704070545.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


0.5329717728599311
              precision    recall  f1-score   support

           0       0.90      0.99      0.94     59984
           1       0.93      0.60      0.73      7541
           2       0.96      0.43      0.59      2833
           3       0.94      0.43      0.59       477
           4       0.96      0.49      0.65      1925
           5       0.97      0.41      0.58       605
           6       0.99      0.40      0.57       662
           7       0.94      0.38      0.54       311

    accuracy                           0.90     74338
   macro avg       0.95      0.52      0.65     74338
weighted avg       0.91      0.90      0.89     74338

0.9041270951599452


In [10]:
train = pd.read_csv('all_tiles_sample9.csv')
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

rf = RandomForestClassifier(n_estimators=10, min_samples_split= 2, min_samples_leaf = 3, max_features=10, max_depth=None, criterion='entropy').fit(X_train, y_train)

y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)
print(log_loss(y_test, y_prob))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

/tmp/ipykernel_24629/1659775143.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


0.5645835338649904
              precision    recall  f1-score   support

           0       0.90      0.99      0.94     59993
           1       0.92      0.58      0.72      7534
           2       0.96      0.40      0.57      2838
           3       0.96      0.42      0.58       476
           4       0.97      0.45      0.61      1925
           5       0.97      0.41      0.58       603
           6       0.99      0.42      0.59       662
           7       0.94      0.32      0.48       307

    accuracy                           0.90     74338
   macro avg       0.95      0.50      0.63     74338
weighted avg       0.90      0.90      0.89     74338

0.9001183782184078


In [11]:
train = pd.read_csv('all_tiles_sample10.csv')
X = train.loc[:, train.columns != 'class']
y = train[['class']]

X = X.rename(columns={x:y for x,y in zip(X.columns,range(0,len(X.columns)))})
X.fillna(0, inplace=True)
y.fillna(0, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y.values.ravel(), test_size=0.3, random_state=42)

rf = RandomForestClassifier(n_estimators=10, min_samples_split= 2, min_samples_leaf = 3, max_features=10, max_depth=None, criterion='entropy').fit(X_train, y_train)

y_pred = rf.predict(X_test)
y_prob = rf.predict_proba(X_test)
print(log_loss(y_test, y_prob))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

/tmp/ipykernel_24629/4256915627.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y.fillna(0, inplace=True)


0.5597835191195029
              precision    recall  f1-score   support

           0       0.90      1.00      0.94     60002
           1       0.93      0.58      0.72      7527
           2       0.96      0.41      0.57      2835
           3       0.96      0.42      0.59       479
           4       0.97      0.45      0.61      1922
           5       0.98      0.42      0.58       604
           6       0.98      0.43      0.60       660
           7       0.96      0.34      0.51       309

    accuracy                           0.90     74338
   macro avg       0.95      0.51      0.64     74338
weighted avg       0.91      0.90      0.89     74338

0.9006968172401733
